<a href="https://colab.research.google.com/github/mfierbaugh/asp-ai-lab/blob/main/asp_ai_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ASP Lab for AI

The ASP Lab for AI consists of 3 different labs. 

* Lab 1 - OpenAI Introductory Lab
* Lab 2 - Local VectorDB Lab
* Lab 3 - Retrieval Augmented Generation

## Pre-Lab Setup

### Step 1: Clone the github repository

Clone the gitbub repo that contains all of the files we will use during the course of this lab.

In [ ]:
!git clone https://github.com/mfierbaugh/asp-ai-lab.git

### Step 2: Enter the OpenAI API Key

Enter the OpenAI API Key.  This will be provided to you on Webex Teams

### Step 3: Install the required Python models for this lab

In [ ]:
!pip install langchain langchain-community langchain-chroma langchain-core langchain-experimental langchain-openai pypdf sentence_transformers openai

## Lab 1: OpenAI Introductory Lab

This lab is a walk through of how to build your first chatbot using openAI.  

### Import Python Libraries

This lab uses openAI's Chat Completion API

Now that we have the openAI libraries installed, we are going to import those libraies into our code so we can use them. We are also going to add the google.colab library so that we can retrieve our openai API key.

In [ ]:
from openai import OpenAI
from google.colab import userdata

We have the ability to use different models in openAI's API.  Let's create a simple variable and store the model name as a string.  We will pass this to openAI via a function later.  

In [ ]:
model = "gpt-3.5-turbo"

This is a function in python that will have 3 things passed to it when we call it.  

1. The user's question.
2. The client - in our case OpenAI with the API key defined.
3. The model we want to use.

Coding up the function in this way alows for us to change the model or client if we so desire.  

This is using OpenAI's Chat Completions API and contains the inputs along with the instructions for providing an output.

### Functions

We are only using a single function in this lab.  It is all we will need.  

The entire prompt is assembled here, including the user's question and anything we combine with it.

In [ ]:
def chat_openai (user_question, client, model):
    """
    This function sends a chat message to the OpenAI API and returns the content of the response.
    It takes two parameters: the chat prompt and the model to use for the chat.
    """
    prompt = """
    {user_question}

    Analyze the user's question and provide an answer based upon the context of the question.
    """.format(
        user_question=user_question
    )

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "You are a rude cynical assistant network engineer:",
            },
            {"role": "user", "content": prompt},
        ],
    )
    return response.choices[0].message.content

The Chat Completions API supports text and image inputs, and can output text content (including code and JSON).

Let's take a closer look at the prompts.

The prompt includes the user question and a set of instructions designed to control the output of the response.  

Each message object has a role (either system, user, or assistant) and content.

*   The system message is optional and can be used to set the behavior of the assistant
*   The user messages provide requests or comments for the assistant to respond to
*   Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior



Notice above how we have set the behavior of the assistant by telling it "You are a rude cynical assistant network engineer:"

We are just having some fun here.  We can make the responses come back snarky, just to show how we can change the behavior.

Now, let's create a the user's question.  You can change this if you like.

In [ ]:
user_question = "Which routers are better, Juniper or Cisco?"

Now, We are going to send the system prompt and the user question to openAI's API using the function we defined above.

In [ ]:
openai_api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=openai_api_key)

response = chat_openai(user_question, client, model)

print(response)

🏆 Congratulations!  Lab 1 is complete

## Lab 2: Local VectorDB Lab

This lab will demonstrate how we can create a local vector database and store unstructured data.  We will then use a semantic search of the data that is stored.

First we will import all of libraries that we will need for this part of the lab. 

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_chroma import Chroma

We are going to take some local documents that we pulled from the github repository for this lab.  These are PDF files and are now stored in your local runtime enviornment. We will set the "file_dir" variable to a string that represents the relative path to the directory where these are stored.  

In [ ]:
file_dir = 'asp-ai-lab/files'
embedding_model = 'all-MiniLM-L6-v2'

The Langchain DirectoryLoader let's us load all of the files in the specified file directory.  We will use the variable defined above and pass that to the DirectoryLoader. 

We then load the docs in that directory with DirectoryLoader's load() function. 

In [ ]:
loader = DirectoryLoader(file_dir, use_multithreading=True, loader_cls=PyPDFLoader)
documents = loader.load()

Now that we have the documents loaded, want to split the text from the document into chunks. 

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=5)
docs = text_splitter.split_documents(documents)

We will use OpenAI's embeddings model.  Embeddings are a vector list of floating point numbers. They allow us to measure how things are related by giving them a floating point number and measuring the distance between them.  

Embeddings are commonly used for:
* Search (where results are ranked by relevance to a query string)
* Clustering (where text strings are grouped by similarity)
* Recommendations (where items with related text strings are recommended)
* Anomaly detection (where outliers with little relatedness are identified)
* Diversity measurement (where similarity distributions are analyzed)
* Classification (where text strings are classified by their most similar label)

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

Chroma is an AI-native open-source vector database. After splitting the docs into chunks, we will use the OpenAI embeddings and insert them into a collection. 

In [ ]:
db = Chroma.from_documents(docs, embeddings)

The query variable contains the question we will ask on the docs.  doc_search runs the similarity search on the Chroma vector database. 

In [ ]:
query = 'How many 400 GigE interfaces can the PTX10002-36QDD support?'
doc_search = db.similarity_search(query)

Printing the result, we should see some chunks that we think are the most similar to the question. 

In [ ]:
print (doc_search[0].page_content)

🏆 Congratulations!  Lab 2 is complete

## Lab 3: OpenAI LAB - Retrieval Augmented Generation

This lab combines the previous two labs and will show how we can retrieve information locally and augment the input.  Then we will specifically instruct the OpenAI Chat Completions API (prompt) to answer the question based upon the provided data.  This is an example of RAG (Retrieval Augmented Generation)

### Import Python Libraries

Import the required libraries for our lab

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_chroma import Chroma
from openai import OpenAI
from google.colab import userdata

### Load the Local Vector Database (Chroma)

This is the same as we did in lab 2 execpt we are loading the docs, doing the embeddings, splitting, and inserting into the vectordb all in a single function. 

In [ ]:
def load_vectordb():
    # load the documents from the directory
    loader = DirectoryLoader(file_dir, use_multithreading=True, loader_cls=PyPDFLoader)
    documents = loader.load()
    # split the documents into chunks
    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
    text_splitter = SemanticChunker(embeddings=embeddings)
    docs = text_splitter.split_documents(documents)

    # store the documents and embeddings in the database
    db = Chroma.from_documents(docs, embeddings)
    return db

### Query the local vector database (Chroma)

This is a python function to query our vector database. When we call this, we will pass it the db and the query string.  

What we are doing differently this time is that we are using a similarity search with scores.  Since it measures the distance (similarity), the lower the number (shorter the distance), the more similar it is.

By default Chroma returns 4 documents, but we restrict it to return only documents that meet a relevance score threshold. 

In [ ]:
def query_vectordb(query, db):
    # query the database
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5})
    docs = retriever.invoke(query)
    return docs

### Create the Prompt for the User Question

You will see that we are going to combine the user question with the retrieved data. The context will be the information we retrieved locally from the vector database.  

In [ ]:
def create_prompt(query, context):
    prompt = f"""
    Use the following pieces of context to answer the question at the end.
    If you do not know the answer, please think rationally and answer from your own knowledge base.

    {context}

    Question: {query}
    """
    return prompt

### OpenAI Chat Completions API


This is very similar to what we used in lab 1.  Now we are a helpful network engineer rather than a rude cynical one.

In [ ]:
def chat_openai (user_question, client, model):
    """
    This function sends a chat message to the OpenAI API and returns the content of the response.
    It takes two parameters: the chat prompt and the model to use for the chat.
    """
    prompt = """
    {user_question}

    Analyze the user's question and provide an answer based upon the context of the question.
    """.format(
        user_question=user_question
    )

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "You are a helpful network engineering expert:",
            },
            {"role": "user", "content": prompt},
        ],
    )
    return response.choices[0].message.content

### Run the application

First, let's just ask OpenAI to give us some information about the device.  Remember, given that the training data stopped before this network device was available or that there was any information about it, gpt-3.5-turbo should have no idea what this actually is. 

Note - what happened? 

In [ ]:
question_nodb = 'How many 400 GigE interfaces can the PTX10002-36QDD support?'
response = chat_openai(question_nodb, client, model)

print(response)

OpenAI likely tried to answer the question and made things up.  This is what is known as a hallucination. 

Now we are going to show the difference.  

We have a foundation model (gpt-3.5-turbo) - like an untrained employee. 
We are going to load the vector database with the pdfs after they have been chunked and embeddings done.  
We are going to do a cosine similarity search, with a threshold. 
We are going to add that to the user's question and send it to the foudnation model.  

No fine-tuning. Just the model + the vectorstore data.

Behold, the power of RAG...

In [ ]:
db = load_vectordb()
query = 'How many 400 GigE interfaces can the PTX10002-36QDD support?'
docs = query_vectordb(query, db)
# print the results
query_result = (docs[0].page_content)
prompt = create_prompt(query, query_result)
client = OpenAI(api_key=openai_api_key)
response = chat_openai(prompt, client, model)

print(response)

🏆 Congratulations!  Lab 3 is complete